In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\lapto\\OneDrive\\Desktop\\Face-reg'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
from src.Facerecognition.constants import *
from src.Facerecognition.utils.common import read_yaml, create_directories
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from src.Facerecognition.entity.config_entity import DataprocessConfig

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    
    
    def get_data_processing_config(self) -> DataprocessConfig:
        config = self.config.data_preprocessing
        data = os.path.join(self.config.data_ingestion.unzip_dir, "Facerecognition")
        create_directories([Path(config.root_dir)])
        
        data_preprocess_config = DataprocessConfig(
            root_dir=Path(config.root_dir),
            preprocessed_dir=config.preprocessed_dir,
            data = Path(data)
        )

        return data_preprocess_config    
    
    def get_training_config(self, data_preprocess_config: DataprocessConfig) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        training_data = data_preprocess_config.root_dir / "preprocessed_images"

        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=self.params.EPOCHES,
            params_batch_size=self.params.BATCH_SIZE,
            params_image_size=self.params.IMAGE_SIZE
        )

        return training_config


In [7]:
import time

In [15]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks(),  # Call the method to get a TensorBoard callback object
            self._create_ckpt_callbacks()
        ]


In [16]:
# class PrepareCallback:
#     def __init__(self, config: PrepareCallbacksConfig):
#         self.config = config


    
#     @property
#     def _create_tb_callbacks(self):
#         timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
#         tb_running_log_dir = os.path.join(
#             self.config.tensorboard_root_log_dir,
#             f"tb_logs_at_{timestamp}",
#         )
#         return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    


#     def _create_ckpt_callbacks(self):
#         return tf.keras.callbacks.ModelCheckpoint(
#             filepath=self.config.checkpoint_model_filepath,
#             save_best_only=True
#         )


#     def get_tb_ckpt_callbacks(self):
#         return [
#             self._create_tb_callbacks,
#             self._create_ckpt_callbacks
        # ]

In [17]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from tensorflow.keras.applications.mobilenet import preprocess_input

In [18]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            preprocessing_function=preprocess_input,
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


        train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [19]:
# ...

try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    data_preprocess_config = config.get_data_processing_config()  # Corrected method name

    training_config = config.get_training_config(data_preprocess_config)  # Pass the data_preprocess_config

    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e


[2023-10-13 15:47:49,887: INFO: common yaml file: config\config.yaml loaded successfully]
[2023-10-13 15:47:49,889: INFO: common yaml file: params.yaml loaded successfully]
[2023-10-13 15:47:49,890: INFO: common created directory at: artifacts]
[2023-10-13 15:47:49,891: INFO: common created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-10-13 15:47:49,891: INFO: common created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2023-10-13 15:47:49,892: INFO: common created directory at: artifacts\data_preprocessing]
[2023-10-13 15:47:49,893: INFO: common created directory at: artifacts\training]
Found 6 images belonging to 4 classes.
Found 30 images belonging to 4 classes.
30/30 [==============================] - 6s 178ms/step - loss: 1.3426 - accuracy: 0.3667 - val_loss: 0.8999 - val_accuracy: 1.0000


c:\Users\lapto\OneDrive\Desktop\Face-reg\regenv\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
